In [118]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import os
import json
import re
import datetime

In [119]:
def adicionar_categoria(categoria):
    if os.path.isfile('ControleArquivos.json'):
        with open('ControleArquivos.json', 'r') as controle:
            data = json.load(controle)
        if categoria not in data:
            data[categoria] = {
                "diretorio": os.path.join(os.getcwd(), os.path.abspath('..\\'), 'Bases', categoria),
            }
            if not os.path.isdir(data[categoria]["diretorio"]):
                os.makedirs(data[categoria]["diretorio"])

        with open('ControleArquivos.json', 'w') as controle:
            json.dump(data, controle, indent=4)

def retornar_dados():
    if os.path.isfile('ControleArquivos.json'):
        with open('ControleArquivos.json', 'r') as controle:
            data = json.load(controle)
        return data
    else:
        return None

def retornar_meses_pendentes(categoria, ano):
    meses = range(1, 13)
    dados = retornar_dados()
    meses_inseridos = set(int(item['mes']) for item in dados[categoria]['arquivos'] if item['ano'] == str(ano))
    if ano == datetime.datetime.now().year:
        meses = range(1, datetime.datetime.now().month + 1)

    meses_pendentes = [mes for mes in meses if mes not in meses_inseridos]
    return meses_pendentes

def retornar_arquivos_pendentes():
    pendentes = {}
    for categoria in categorias:
        adicionar_categoria(categoria)
        pendentes[categoria] = {}
        for ano in anos:
            meses = retornar_meses_pendentes(categoria, ano)
            if meses:
                pendentes[categoria][ano] = {'meses': meses}

        if not pendentes[categoria]:
            pendentes.pop(categoria)
    return pendentes



In [120]:
botoes = {
    "categoria": "cphBody_btn",
    "ano": "cphBody_lkAno",
    "mes": "cphBody_lkMes",
    "exportar": "cphBody_ExportarBOLink"
}

def abrir_navegador():
    opcoes = Options()
    # options.add_argument('--headless')
    navegador = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=opcoes)
    navegador.get('http://www.ssp.sp.gov.br/transparenciassp/Consulta.aspx')
    return navegador

def clicar_botao(botao, valor):
    elemento = (By.XPATH, f'//*[@id="{botoes[botao]}{valor}"]')
    WebDriverWait(navegador, 30).until(EC.element_to_be_clickable(elemento)).click()


In [121]:
categorias = [
    'FurtoCelular',
    'RouboCelular'
]

anos = range(2010, datetime.datetime.now().year + 1)

In [ ]:
arquivos_pendentes = retornar_arquivos_pendentes()

if arquivos_pendentes == {}:
    print('Não há arquivos pendentes para baixar')
    exit()

navegador = abrir_navegador()
for categoria in arquivos_pendentes:
    clicar_botao('categoria', categoria)
    for ano in arquivos_pendentes[categoria]:
        clicar_botao('ano', ano % 100)
        for mes in arquivos_pendentes[categoria][ano]['meses']:
            clicar_botao('mes', mes)
            clicar_botao('exportar', '')



In [ ]:
# teste
navegador = abrir_navegador()
clicar_botao('categoria', 'FurtoCelular')
clicar_botao('ano', '10')
clicar_botao('mes', '7')
clicar_botao('exportar', '')

clicar_botao('mes', '8')
clicar_botao('exportar', '')

clicar_botao('categoria', 'RouboCelular')
clicar_botao('ano', '10')
clicar_botao('mes', '2')
clicar_botao('exportar', '')

clicar_botao('mes', '3')
clicar_botao('exportar', '')